In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
import sklearn

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', as_frame=False)


/Users/pgrigorakis/Documents/HOML/homl-venv/lib/python3.9/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [11]:
X, y = mnist.data, mnist.target
X_train, X_test, y_train, y_test =  X[:60000], X[60000:], y[:60000], y[60000:]

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
cross_val_score(knn_clf, X_train, y_train, cv=3, scoring="accuracy")


KNeighborsClassifier()

In [26]:
from sklearn.model_selection import GridSearchCV

parameters = {'weights':('uniform', 'distance' ), 'n_neighbors':[3,5,7,9]}
knn = KNeighborsClassifier()
knn_gsv = GridSearchCV(knn, parameters, cv=3, scoring='accuracy')
knn_gsv.fit(X, y)
knn_gsv.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [30]:
grid_cv_res = pd.DataFrame(knn_gsv.cv_results_)
grid_cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
grid_cv_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1,0.115005,0.004538,10.702452,0.260479,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.971029,0.969057,0.969571,0.969886,0.000836,1
0,0.116424,0.007814,9.306698,0.441554,3,uniform,"{'n_neighbors': 3, 'weights': 'uniform'}",0.969529,0.968114,0.968500,0.968714,0.000598,2
3,0.120893,0.003772,11.727547,0.166338,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.969572,0.967042,0.969057,0.968557,0.001092,3
5,0.123318,0.010926,12.111456,0.206436,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.969101,0.966357,0.967985,0.967814,0.001127,4
2,0.113079,0.007755,12.245364,0.593749,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.968372,0.966185,0.967814,0.967457,0.000928,5
4,0.116114,0.010245,12.289315,0.105882,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.967901,0.965242,0.967000,0.966714,0.001104,6
7,0.127257,0.006666,12.264792,0.030658,9,distance,"{'n_neighbors': 9, 'weights': 'distance'}",0.967258,0.964471,0.967428,0.966386,0.001356,7
6,0.116536,0.009150,12.595843,0.076359,9,uniform,"{'n_neighbors': 9, 'weights': 'uniform'}",0.966058,0.962885,0.966142,0.965029,0.001516,8


In [35]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'weights': ('uniform', 'distance'), 'n_neighbors':randint(low=1, high=20)}
knn_random_search  = RandomizedSearchCV(knn, param_distributions=param_distribs, n_iter=10, cv=3, scoring="accuracy", random_state=42)
knn_random_search.fit(X_train, y_train)
knn_random_search_res = pd.DataFrame(knn_random_search.cv_results_)
knn_random_search_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
knn_random_search_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
6,0.105093,0.006689,9.167126,0.296519,4,distance,"{'n_neighbors': 4, 'weights': 'distance'}",0.97090,0.96980,0.97035,0.970350,0.000449,1
7,0.113930,0.000240,8.956183,0.178719,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.97040,0.96890,0.96870,0.969333,0.000759,2
8,0.111286,0.000206,9.292564,0.203294,2,distance,"{'n_neighbors': 2, 'weights': 'distance'}",0.96875,0.96685,0.96705,0.967550,0.000852,3
0,0.091854,0.001274,6.493022,0.136543,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.96700,0.96560,0.96740,0.966667,0.000772,4
3,0.099635,0.003523,8.687911,0.179160,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.96700,0.96560,0.96740,0.966667,0.000772,4
2,0.092557,0.001638,7.982436,0.275446,8,uniform,"{'n_neighbors': 8, 'weights': 'uniform'}",0.96365,0.96370,0.96570,0.964350,0.000955,6
9,0.110205,0.000546,9.664323,0.293897,12,distance,"{'n_neighbors': 12, 'weights': 'distance'}",0.96215,0.96290,0.96515,0.963400,0.001275,7
5,0.096318,0.004342,8.824394,0.021875,11,uniform,"{'n_neighbors': 11, 'weights': 'uniform'}",0.96110,0.96140,0.96365,0.962050,0.001138,8
1,0.093922,0.001502,7.306141,0.137320,15,uniform,"{'n_neighbors': 15, 'weights': 'uniform'}",0.95990,0.95910,0.96175,0.960250,0.001110,9
4,0.102724,0.003236,8.718237,0.017182,19,uniform,"{'n_neighbors': 19, 'weights': 'uniform'}",0.95810,0.95655,0.95955,0.958067,0.001225,10


In [65]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, precision_recall_curve 
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
pd.options.display.max_rows = 4000

best_model = knn_random_search.best_estimator_ 
knn_predictions = best_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_f1 = f1_score(y_test, knn_predictions, average='macro')
knn_precision = precision_score(y_test, knn_predictions, average='micro')
knn_recall = recall_score(y_test, knn_predictions, average='micro')
knn_cm = confusion_matrix(y_test, knn_predictions)
knn_class_report = classification_report(y_test, knn_predictions, output_dict=False,
                                         target_names=['0','1','2','3','4','5','6','7','8','9','10'])

print(f"Accuracy is: {knn_accuracy},\nPrecision is {knn_precision},\nRecall is {knn_recall},\nf1-score is {knn_f1}\n")
print(f"Confussion Matrix: \n\n {knn_cm} \n")
print(f"Classification Report: \n\n {knn_class_report} \n")
# knn_fpr, knn_tpr, knn_thresholds = roc_curve_score(y_test, knn_predictions, pos_label='label')
#precisions, recalls, thresholds = precision_recall_curve(y_test, knn_predictions)
# plt.plot(knn_fpr, knn_tpr, linewidth=2, label='roc_curve')
# plt.show()